
excerpt 分析结果合并

文件夹格式:
    
    GSE_ID_out/SRR_ID/readCounts_miRNAmature_sense.txt
    
合并思路：

    和miRbase miRNA 列表合并，以其为索引的中间文件，文件名为SRR_ID
    
    合并中间文件，列名为SRR_id
    
需要文件：
    
    miRBASE 记录的miRNA 列表
    
    SRR_list
    
注意事项：

    来源不同的相同成熟miRNA, 存在于不同的行
    
备注：服务器上完成

In [1]:
import pandas as pd
import os

In [2]:
df_mir_ = pd.read_csv('miR_list.txt',sep='\t') # ['miRNAID'].to_list()

GSE_ID = 'GSE110907'

file_path = f'{GSE_ID}_out'

mir_file = 'readCounts_miRNAmature_sense.txt'

with open(f'{GSE_ID}/{GSE_ID}_SRR_Acc_List.txt','r') as f:
    sra_list = [srr.replace('\n','') for srr in f.readlines()]

In [4]:
df_ = pd.read_csv(os.path.join(file_path, 'SRR6757422', mir_file),sep='\t')[['ReferenceID','totalReadCount']]
print(len(set(df_['ReferenceID'])))
df_['miRNA_ID'] = df_['ReferenceID'].str.split(':')
df_['miRNA_ID'] = df_['miRNA_ID'].str[0]
len(set(df_['miRNA_ID']))

394


392

In [5]:
df_.groupby('miRNA_ID')['totalReadCount'].sum().reset_index()

,miRNA_ID,totalReadCount
0,hsa-let-7a-3,5
1,hsa-let-7a-5p,518122
2,hsa-let-7b-5p,674
3,hsa-let-7c-5p,518117
4,hsa-let-7d-5p,10749
...,...,...
387,hsa-mir-30c-1,51833
388,hsa-mir-30c-2,9
389,hsa-mir-378a,1
390,hsa-mir-378c,517


In [7]:
import os
for sra in sra_list: # 生成中间文件，列为mirbase21的列

    df_expr_ = pd.read_csv(os.path.join(file_path, sra, mir_file),sep='\t')
    df_expr_['miRNA_ID'] = df_expr_['ReferenceID'].str.split(':')
    df_expr_['miRNA_ID'] = df_expr_['miRNA_ID'].str[0]
    df_expr_mature = df_expr_.groupby('miRNA_ID')['totalReadCount'].sum().reset_index()
    df_tmp = pd.merge(df_mir_, df_expr_mature, left_on = 'miRNAID', right_on='miRNA_ID',how='left')
    df_tmp.drop('miRNA_ID',axis=1,inplace=True)
    df_tmp.fillna(0,inplace=True)
    
    df_tmp.rename(columns={'totalReadCount':sra},inplace=True) # 重命名为 sra_id
    df_tmp.to_csv(f'./{GSE_ID}_tmp/{sra}.csv',index=None)

**合并数据**

In [8]:
tmp_path = f'./{GSE_ID}_tmp/'
mir_files = os.listdir(tmp_path)
mir_df = pd.read_csv(os.path.join(tmp_path,mir_files[0]))
mir_df.head()

,miRNAID,SRR6757403
0,hsa-let-7a-2-3p,0.0
1,hsa-let-7a-3p,0.0
2,hsa-let-7a-5p,983705.0
3,hsa-let-7b-3p,0.0
4,hsa-let-7b-5p,535.0


In [10]:
for mir in mir_files[1:]:
    mir_df_ = pd.read_csv(os.path.join(tmp_path,mir))[mir.replace('.csv','')] # 本数据中miR注释总数相同，合并后顺序对应
    mir_df = pd.concat([mir_df,mir_df_], axis=1)

print(f'miRNA 数据维度：{mir_df.shape}\n')

mir_df.to_csv(f'./{GSE_ID}_result/{GSE_ID}_counts.tsv',sep='\t',index=None)


miRNA 数据维度：(2652, 97)

